# Assignment 2: NLP Basics
**Due date: November 26, 23:55**

## Section 1: GloVe Embeddings
In this section, you will load the traditional GloVe embeddings, and explore the basic properties of the embedding space.

In [ ]:
# Download the GloVe text files
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

### Load the GloVe embeddings [5 pts]

In [8]:
# Load GloVe word vectors into a dictionary
import numpy as np

def load_glove_embeddings(glove_file_path):
    embeddings_dict = {}
    with open(glove_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings_dict[word] = vector
    return embeddings_dict

glove_file_path = 'glove.6B/glove.6B.100d.txt'  # Replace with the path to your GloVe file
glove_embeddings = load_glove_embeddings(glove_file_path)
print(f'Loaded {len(glove_embeddings)} word vectors')
print(glove_embeddings['the'])

Loaded 400000 word vectors
[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0

### Find closest words [5 pts]
For a given word, you should output the similar words and their similarity values.

In [9]:
# Function to find the closest word, and the corresponding similarity value
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def find_closest_word(word, embeddings_dict, top_n=1):
    word_embedding = embeddings_dict[word]
    closest_words = []
    for w, emb in embeddings_dict.items():
        if w != word:
            closest_words.append((w, cosine_similarity(word_embedding, emb)))
    closest_words.sort(key=lambda x: x[1], reverse=True)
    return closest_words[:top_n]
    
chosen_word = 'man'
closest_word = find_closest_word(chosen_word, glove_embeddings, top_n=5)
print(f"The words closest to '{chosen_word}' are:")
for word, similarity in closest_word:
    print(f"{word} with similarity of {similarity:.4f}")

The words closest to 'man' are:
woman with similarity of 0.8323
boy with similarity of 0.7915
one with similarity of 0.7789
person with similarity of 0.7527
another with similarity of 0.7522


### Find new analogies [5 pts]
In the lecture, we discussed how linear relationships exist in the embedding space (e.g. king - man + woman = queen). Please demonstrate an analogy that is not mentioned in the lecture. Feel free to alter the previous function if necessary.

In [10]:
word1 = "paris"
word2 = "france"
word3 = "italy"

def find_analogy(word_a, word_b, word_c, embeddings_dict, top_n=1):
    if word_a not in embeddings_dict or word_b not in embeddings_dict or word_c not in embeddings_dict:
        print(f"Error: One or more words not found in embeddings.")
        return []
    
    analogy_vector = (
        np.array(embeddings_dict[word_a]) 
        - np.array(embeddings_dict[word_b]) 
        + np.array(embeddings_dict[word_c])
    )
    
    similarities = []
    for other_word, other_vector in embeddings_dict.items():
        if other_word in {word_a, word_b, word_c}:
            continue
        similarity = cosine_similarity(analogy_vector, other_vector)
        similarities.append((other_word, similarity))
        
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

print(find_analogy(word1, word2, word3, glove_embeddings, top_n=1))

[('rome', 0.8084002)]


## Section 2: BPE tokenizer

Byte Pair Encoding(BPE) is a subword tokenization technique that iteratively merges the most frequent adjacent byte pairs into subword units, creating a vocabulary that balances character-level granularity and whole-word tokens. This method is widely used in modern natural language processing to handle out-of-vocabulary words and optimize tokenization efficiency.

Let's look at an example. Given a sample string "banana bandana", we can calculate the frequency of the character pairs: 
```python
('a', 'n'): 4, ('n', 'a'): 3, ('b', 'a'): 2, ('a', ' '): 1, (' ','b'): 1, ('n', 'd'): 1, ('d', 'a'): 1
```
Which means that we can combine 'an' into a new token. In the next round, 'an' can now participate in the frequency count, giving:
```python
('b', 'an'): 2, ('an', 'a'): 2, ('an', 'an'): 1, ('a', ' '): 1, (' ','b'): 1, ('an', 'd'): 1, ('d', 'an'): 1
```
So we may get 'ban' as a new token. Similarly, 'ana' would be the most frequent pair in the next round. With three merges, we've added 'an', 'ban' and 'ana' into our vocabulary, and our string can now be converted to the following tokens:
```python
'ban', 'ana', ' ', 'ban','d' ,'ana'
```
So now we can use 6 tokens to represent the 14 characters.

You may wonder how this is better than word-level tokenization. First of all, it is more robust in out-of-vocabulary scenarios. For example, though the word "bandana" does exist in the GloVe embedding (look it up if you're not convinced), something like "banada" does not. When using GloVe embeddings, encountering "banada" during training would result in the default \<UNK\> token. In contrast, a BPE tokenizer can still infer the word's meaning through its sub-word tokens. Secondly, sub-word tokens include prefixes and suffixes that allow the model to learn different variations of a single word more efficiently.

In this section, you are required to implement a BPE tokenizer, and use one of the provided corpora to train it. You may train it on character level (starting with a vocabulary of all characters in the corpus) or byte level (starting with a vocabulary of all 256 possible byte values). You should verify that encoding and then decoding a sentence produces the original sentence. You may refer to (but not copy) the following implementations:
1. The tiktoken library: https://github.com/openai/tiktoken/blob/main/tiktoken/_educational.py
2. Kaparthy's minbpe repository: https://github.com/karpathy/minbpe

### Implementation and Verification [15 pts]

In [ ]:
from collections import defaultdict

class BPETokenizer:
    def __init__(self):
        self.vocab = []
        self.merges = []
    
    def train(self, text: str, vocab_size: int):
        """
        Train the BPE tokenizer on the given text.
        
        :param text: The input text to train the tokenizer.
        :param vocab_size: The desired size of the final vocabulary.
        """
        # Initialize the vocabulary with single characters
        tokens = list(text)
        self.vocab = list(set(tokens))
        token_freq = defaultdict(int)
        while(len(self.vocab) < vocab_size):
            token_freq.clear()
            for i in range(len(tokens) - 1):
                token_freq[(tokens[i], tokens[i + 1])] += 1
            
            most_frequent = max(token_freq, key=token_freq.get)
            new_token = ''.join(most_frequent)
            self.merges.append(most_frequent)
            
            new_tokens = []
            i = 0
            while i < len(tokens):
                if i < len(tokens) - 1 and tokens[i] == most_frequent[0] and tokens[i + 1] == most_frequent[1]:
                    new_tokens.append(new_token)
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
            self.vocab.append(new_token)
        with open('vocab.txt', 'w', encoding='utf-8') as file:
            for token in self.vocab:
                file.write(token + '\n')
    
    def encode(self, text: str) -> list[int]:
        """
        Encode the input text using the learned BPE merges.
        
        :param text: The input text to encode.
        :return: The list of tokens after encoding.
        """
        tokens = list(text)
        for merge in self.merges:
            new_tokens = []
            i = 0
            while i < len(tokens):
                if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == merge:
                    new_tokens.append(''.join(merge))
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
        return new_tokens
    
    def decode(self, tokens: list[int]) -> str:
        """
        Decode the input tokens using the learned BPE merges.
        
        :param tokens: The input tokens to decode.
        :return: The decoded text.
        """
        text = ''.join(tokens)
        return text

In [ ]:
with open("dataset/tinyshakespeare.txt", "r", encoding="utf-8") as file:
    train_text = file.read()
    # print(train_text[:1000])
tokenizer = BPETokenizer()
tokenizer.train(train_text, vocab_size=512)

test_string = "lorem ipsum dolor sit amet, consectetur adipiscing elit."
print(tokenizer.encode(test_string))
assert(tokenizer.decode(tokenizer.encode(test_string))==test_string)
print("Voila!")

['lor', 'em', ' ', 'i', 'p', 'su', 'm', ' d', 'ol', 'or', ' s', 'it ', 'am', 'e', 't, ', 'con', 'se', 'ct', 'et', 'ur', ' a', 'di', 'p', 'is', 'c', 'ing ', 'e', 'li', 't', '.']
Voila!


### Question: In your training process, which full words were the first to be merged into tokens? [5 pt]

Answer: on, is, you, to, and, the are the first several words to be merged into tokens. The common prepositions and pronouns are often the first to be merged.

## Section 3: Text generation

In this section, you will implement an RNN/LSTM model to generate sentences that mimic the style of the chosen corpus. You are free to use additional packages or modify the parameters of the provided function templates as needed.

In [1]:

# Load necessary packages. Feel free to add ones that you need.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import wandb
from typing import Union
import tqdm
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

### Load and preprocess text [5 pts]
You can choose a training corpus from the provided texts. Though the texts are much cleaner than random web crawls, you may still want to perform some preprocessing.

In [2]:
# Load the text file
with open("dataset/trump_2016_speeches.txt", "r", encoding="utf-8-sig") as file:
    text = file.read()

# Preprocess the text (tokenize, remove special characters, etc.)
def preprocess_text(text: str):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

tokens = preprocess_text(text)
print(tokens[:10])

['speech', '1', '...', 'thank', 'you', 'so', 'much', '.', 'that', "'s"]


### Build vocabulary, and setup embedding matrix [5 pts]
You may limit your vocabulary to words in the training corpus instead of using all 40k words in the GloVe embedding. Then, you may assign indices to each word and convert the embedding dictionary into a matrix.

In [3]:
# Build Vocabulary
def build_vocabulary(tokens):
    word_to_idx = {"<UNK>": 0}
    idx_to_word = {0: "<UNK>"}
    idx = 1
    for token in tokens:
        if token not in word_to_idx:
            word_to_idx[token] = idx
            idx_to_word[idx] = token
            idx += 1
    return word_to_idx, idx_to_word

# Load embeddings to matrix
def load_glove_embeddings(glove_file, vocab):
    embedding_matrix = None
    embedding_dim = 0
    with open(glove_file, 'r', encoding='utf-8') as file:
        embeddings_dict = {}
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings_dict[word] = vector
            embedding_dim = len(vector)
        embedding_matrix = np.zeros((len(vocab), embedding_dim), dtype=np.float32)
        for word, idx in vocab.items():
            if word in embeddings_dict:
                embedding_matrix[idx] = embeddings_dict[word]
            else:
                # all map to the <UNK> token with index 0
                continue

    return torch.tensor(embedding_matrix)

### Implement the dataset [10 pts]
The text generation task uses next word prediction as its objective. Think about how to construct your training data.

In [4]:
# Construct your dataset
class TextDataset(Dataset):
    def __init__(self, text, context_size, vocab):
        """
        Construct a Dataset for next-word prediction.
        
        :param text: List of tokens (words) from the corpus.
        :param context_size: Number of words to use as context for predicting the next word.
        :param vocab: Dictionary mapping words to indices.
        """
        self.context_size = context_size
        self.vocab = vocab

        # Preprocess the data to generate context-target pairs
        self.data = []
        for i in range(context_size, len(text)):
            context = text[i - context_size:i]
            target = text[i]
            context_indices = [vocab[word] for word in context if word in vocab]
            target_index = vocab[target]

            # Ensure the context length matches context_size
            if len(context_indices) == context_size:
                self.data.append((context_indices, target_index))

    def __len__(self):
        """
        Return the total number of samples.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Get a single context-target pair.
        
        :param idx: Index of the data point.
        :return: Tuple (context_tensor, target_tensor).
        """
        context, target = self.data[idx]
        context_tensor = torch.tensor(context, dtype=torch.long)
        target_tensor = torch.tensor(target, dtype=torch.long)
        return context_tensor, target_tensor

### Implement the RNN(LSTM) model [10 pts]
You are allowed to use nn.LSTM in this section. Feel free to try different model architectures as well.

In [5]:
# Construct your model
class TextGenLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix):
        """
        Initialize the TextGenLSTM model.

        :param vocab_size: Size of the vocabulary.
        :param embedding_dim: Dimension of word embeddings.
        :param hidden_dim: Dimension of LSTM hidden states.
        :param num_layers: Number of LSTM layers.
        :param embedding_matrix: Pre-trained embedding matrix.
        """
        super(TextGenLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x): # x: (batch_size, seq_length)
        embeddings = self.embedding(x) # (batch_size, seq_length, embedding_dim)
        out, _ = self.lstm(embeddings) # out: (batch_size, seq_length, D * hidden_dim)
        out = self.fc(out[:, -1, :])
        return out
    
class TextGenLSTMWithAttn(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, embedding_matrix):
        """
        Initialize the TextGenLSTM with QKV Attention model.

        :param vocab_size: Size of the vocabulary.
        :param embedding_dim: Dimension of word embeddings.
        :param hidden_dim: Dimension of LSTM hidden states.
        :param num_layers: Number of LSTM layers.
        :param embedding_matrix: Pre-trained embedding matrix.
        """
        super(TextGenLSTMWithAttn, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)

        # QKV attention layers
        self.Wk = nn.Linear(hidden_dim, hidden_dim)
        self.Wv = nn.Linear(hidden_dim, hidden_dim)
        self.Wq = nn.Linear(hidden_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):  # x: (batch_size, seq_length)
        embeddings = self.embedding(x)  # (batch_size, seq_length, embedding_dim)

        lstm_out, (hidden, _) = self.lstm(embeddings)

        Q = self.Wq(hidden[-1])  # (batch_size, hidden_dim)

        K = self.Wk(lstm_out)  # (batch_size, seq_length, hidden_dim)
        V = self.Wv(lstm_out)  # (batch_size, seq_length, hidden_dim)

        attention_scores = torch.matmul(Q.unsqueeze(1), K.transpose(-2, -1)) / (self.hidden_dim ** 0.5)  # (batch_size, 1, seq_length)
        attention_weights = F.softmax(attention_scores, dim=-1)  # (batch_size, 1, seq_length)

        context = torch.matmul(attention_weights, V).squeeze(1)  # (batch_size, hidden_dim)

        output = self.fc(context)  # (batch_size, vocab_size)

        return output

### Implement a generate_text function [10 pts]
Even with the same model, you can make it output totally different sentences. Try to make your model generate coherent text.

In [6]:
# Generate text with your model

def generate_text(model, start_sequence, num_words, vocab) -> str:
    """
    Generate text using the trained model.

    :param model: The trained text generation model.
    :param start_sequence: A string representing the starting sequence.
    :param num_words: Number of words to generate.
    :param vocab: A dictionary with word-to-index and index-to-word mappings.
                  Expected structure: {"word2idx": {}, "idx2word": {}}.
    :param context_size: The context size used by the model.
    :return: Generated text as a string.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    word2idx = vocab["word2idx"]
    idx2word = vocab["idx2word"]

    # Tokenize the start sequence
    start_sequence_tokens = preprocess_text(start_sequence)
    input_indices = [word2idx.get(word, word2idx["<UNK>"]) for word in start_sequence_tokens]

    input_tensor = torch.tensor(input_indices, dtype=torch.long).unsqueeze(0).to(device)
    generated_words = start_sequence_tokens[:]
    new_words = []
    
    with torch.no_grad():
        for _ in range(num_words):
            output = model(input_tensor)  # (1, vocab_size)
            predicted_idx = torch.argmax(output, dim=-1).item()
            predicted_word = idx2word[predicted_idx]
            generated_words.append(predicted_word)
            new_words.append(predicted_word)
            input_indices.append(predicted_idx)
            input_tensor = torch.tensor(input_indices, dtype=torch.long).unsqueeze(0).to(device)

    return TreebankWordDetokenizer().detokenize(new_words).capitalize()

### Implement the training loop [10 pts]
Now, you can finally train your model. During each epoch, remember to log appropriate stats, and let the model generate sentences to see how the training progresses. Since we're only using an RNN, there's no need to panic if you don't see GPT-like generation performance.

In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, vocab, start_sequence, epochs, context_size: Union[int, None]):
    """
    Train the model with logging and text generation.

    :param model: The PyTorch model to train.
    :param train_loader: DataLoader for training data.
    :param val_loader: DataLoader for validation data.
    :param criterion: Loss function.
    :param optimizer: Optimizer for updating model weights.
    :param device: Device to run the model on ('cpu' or 'cuda').
    :param vocab: Vocabulary containing word-to-index and index-to-word mappings.
    :param start_sequence: String to initialize text generation.
    :param epochs: Number of epochs to train.
    """
    best_val_loss = float('inf')
    best_model = None

    for epoch in tqdm.tqdm(range(epochs), total=epochs, desc="Epochs"):
        # Training phase
        model.train()
        train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device).to(torch.long), targets.to(device).to(torch.long)
            outputs = model(inputs).to(device)
            loss = criterion(outputs, targets).to(device)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device).to(torch.long), targets.to(device).to(torch.long)
                outputs = model(inputs).to(device)
                loss = criterion(outputs, targets).to(device)
                val_loss += loss.item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model

        # Log results to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "val_loss": val_loss,
        })
        
    sample_text = generate_text(model, start_sequence, 10, vocab)
    print(f"Generated Example: {sample_text}")
    
    return best_model

In [8]:
config = {
    "context_size": 64,
    "glove_file": "glove.6B/glove.6B.300d.txt",
    "hidden_dim": 256,
    "num_layers": 1,
    "batch_size": 64,
    "learning_rate": 5e-4,
    "epochs": 15,
    "model": "lstm", # or "lstm_with_attn"
}

In [ ]:
# initialize training data
word_2_idx, idx_2_word = build_vocabulary(tokens)
dataset = TextDataset(tokens, config["context_size"], word_2_idx)

# Load the GloVe embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_matrix = load_glove_embeddings(config["glove_file"], word_2_idx).to(device)
vocab_size = len(word_2_idx)
embedding_dim = embedding_matrix.shape[1]

# Initialize the data loader
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_data, batch_size=config["batch_size"], shuffle=False)
    
# initialize the model
if config["model"] == "lstm":
    model = TextGenLSTM(vocab_size, embedding_dim, config["hidden_dim"], config["num_layers"], embedding_matrix).to(device)
elif config["model"] == "lstm_with_attn":
    model = TextGenLSTMWithAttn(vocab_size, embedding_dim, config["hidden_dim"], config["num_layers"], embedding_matrix).to(device)
    
# Initialize the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

# initialize wandb
wandb.init(
    project="llm",
    name=f"{config['model']}",
    config={
        "vocab_size": vocab_size,
        "embedding_dim": embedding_dim,
        "hidden_dim": config["hidden_dim"],
        "num_layers": config["num_layers"],
        "batch_size": config["batch_size"],
        "epochs": config["epochs"],
        "learning_rate": config["learning_rate"],
        "context_size": config["context_size"],
    }
)

# Train the lstm
trained_lstm = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    vocab={"word2idx": word_2_idx, "idx2word": idx_2_word},
    start_sequence="Trump is going to be the president of the United States in January next year. This has led to Chinese people worrying about the visa policy.",
    epochs=config["epochs"],
    context_size=config["context_size"], 
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dora23333 (dora23333-Peking University). Use `wandb login --relogin` to force relogin


Epochs: 100%|██████████| 15/15 [02:33<00:00, 10.20s/it]

Generated Example: We have to be vigilant . we have to be
